# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [34]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [29]:
import pandas as pd
weather_data = pd.read_csv("WeatherPy_data.csv")
weather_data


,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [28]:
gmaps.configure(api_key=g_key)
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]
fig = gmaps.figure (center=(50.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity = 100, point_radius=3)
fig.add_layer(heat_layer)
fig

TraitError: The 'locations' trait of a WeightedHeatmap instance must be of length 1 <= L <= 9223372036854775807, but a value of [] was specified.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <=20) & (weather_data["Cloudiness"] ==0) & \
(weather_data["Temp"] >=60) & (weather_data["Temp"] <=70)].dropna( )
narrowed_city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Temp,Lat,Lng,Wind Speed


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = " "
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [31]:
hotellist=[]
for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i] ['Lat']
    lng = narrowed_city_df.loc[i] ['Lng']
    params = {
        "location": f"(lat), (lng)",
        "radius": 5000,
        "types" : "hotel",
        "key" : g_key
    }

In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [33]:
# Add marker layer ontop of heat map


# Display figure
hotel_df

,City,Country,Lat,Lng,Hotel Name
